# **Import Libraries**

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# **Split the Dataset**

In [3]:
# create train df m
df_train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
df_test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [4]:
df_train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB


# **Let's Build the Model**

In [6]:
from sklearn.model_selection import train_test_split
X = df_train.drop('label', axis=1)
y = df_train['label']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=4242)

In [7]:
# reshape the data
X_train = X_train.values.reshape(-1, 28, 28, 1)
X_val = X_val.values.reshape(-1, 28, 28, 1)
X_test = test.values.reshape(-1, 28, 28, 1)
y_val = y_val.values.reshape(-1, 1)
y_train = y_train.values.reshape(-1, 1)

NameError: name 'test' is not defined

# **Using One Hot Encoding**

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
y_train = enc.fit_transform(y_train).toarray()
y_val = enc.fit_transform(y_val).toarray()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1
)

In [ ]:
plt.figure(figsize=(8, 8))
for X_batch, y_batch in datagen.flow(X_train, y_train, batch_size=9):
    for i in range(0, 9):
        plt.subplot(330 + 1 + i)
        plt.imshow(X_batch[i].reshape(28, 28), cmap=plt.get_cmap('gray'))
    plt.show()
    break

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# train the model
history = model.fit_generator(datagen.flow(X_train, y_train, batch_size=128), epochs=20, validation_data=(X_val, y_val))

In [ ]:
# graph the training and validation loss
plt.figure(figsize=(12, 8))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()

In [ ]:
# plot the accuracy
plt.figure(figsize=(12, 8))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.show()

In [ ]:
# CNN model with batch normalization
bn_model = Sequential()
bn_model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
bn_model.add(Conv2D(32, (3, 3), activation='relu'))
bn_model.add(MaxPooling2D())
bn_model.add(tf.keras.layers.BatchNormalization())
bn_model.add(Dropout(0.2))
bn_model.add(Conv2D(64, (3, 3), activation='relu'))
bn_model.add(Conv2D(64, (3, 3), activation='relu'))
bn_model.add(MaxPooling2D())
bn_model.add(tf.keras.layers.BatchNormalization())
bn_model.add(Dropout(0.2))
bn_model.add(Flatten())
bn_model.add(Dense(128, activation='relu'))
bn_model.add(tf.keras.layers.BatchNormalization())
bn_model.add(Dense(10, activation='softmax'))

In [ ]:
# fit the model
bn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
bn_history = bn_model.fit_generator(datagen.flow(X_train, y_train, batch_size=64), epochs=30, validation_data=(X_val, y_val))

In [ ]:
# print the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = bn_model.predict(X_val)
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_val, axis=1)
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(12, 8))
sns.heatmap(cm, annot=True, fmt='d')
# axis labels
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
# Load and preprocess test data
test_data = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
test_data = test_data.values.reshape(-1, 28, 28, 1)
test_data = test_data / 255.0

# Make predictions
predictions = model.predict(test_data)
resized_predictions = predictions.argmax(axis=1).reshape(-1, 1)

In [ ]:
# Assuming resized_predictions is a 2D NumPy array
flat_predictions = resized_predictions.flatten()  # Flatten to a 1D array

# Create a submission DataFrame
submission = pd.DataFrame({'ImageId': range(1, len(test_data) + 1), 'Label': flat_predictions})

# Save the submission to a CSV file
submission.to_csv('test.csv', index=False)

In [ ]:
model.save('model.h5')